In [4]:
import torch.nn as nn
import torch
import numpy as np
import pandas as pd
# from FirstDGL.surrport.dataprocess import dataloader, Arg, my_dataset
# import myfunction

In [5]:
# 结构
class Classifier(nn.Module):
    def __init__(self, arg):
        super(Classifier, self).__init__()
        self.in_dim = arg.input_ft
        self.hidden_dim1 = arg.hidden_dim_1
        self.n_classes = arg.num_labels

        self.conv1 = nn.Conv1d(12, 1, 3, 1, padding=2)
        self.gconv1 = GraphConv(self.in_dim, self.hidden_dim1,
                                norm='none', weight=True, bias=True)
        self.classify1 = nn.Linear(self.hidden_dim1, self.n_classes)
        self.actf = nn.Tanh()

    def forward(self, graph):
        h0 = self.conv1(graph.ndata['h'].float())
        h1 = self.gconv1(graph, h0  # graph.ndata['h'].float()
                         , edge_weight=graph.edata['f'].float())
        with graph.local_scope():
            graph.ndata['h'] = h1
            return self.classify1(hg)  
    
    @property         
    def num_labels(self):
        return 5

    def __getitem__(self, idx):
        return self.graphs[idx], self.label[idx]

    def __len__(self):
        return len(self.graphs)

In [ ]:
arg = dp.Arg()
model = Classifier(arg)

In [ ]:
# main
arg = dp.Arg()
batch_size = arg.batch_size
device = torch.device('cuda:0')
model = Classifier(arg)
model = model.to(device)

opt = torch.optim.Adam(model.parameters(), weight_decay=0.3)

dataset = my_dataset(raw_dir=arg.raw_data_dir, save_dir=arg.save_data_dir)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, test_size])

tr_dataloader = dataloader(train_dataset, arg.batch_size, collate=MF.collate)
ts_dataloader = dataloader(test_dataset, arg.batch_size, collate=MF.collate)

In [ ]:
for epoch in range(1, arg.epoch_num+1):
    # 由于图经过拼合，所以需要多一个dataloader的过程
    loss_list, acc_list = MF.train(
        epoch,
        model, opt, tr_dataloader,
        arg,
        writer
    )

    val_loss, val_acc = MF.evaluate(
        epoch,
        model, opt, ts_dataloader,
        arg,
        writer
    )

In [10]:
import torch
from torch import nn
bn = nn.BatchNorm2d(num_features=3, affine=False, track_running_stats=False)
x = torch.rand(3,2,1,1)*10000
o_bn = bn(x)   # 官方代码
o_bn

tensor([[[[-0.7370]],

         [[ 1.4117]]],


        [[[ 1.4138]],

         [[-0.6331]]],


        [[[-0.6768]],

         [[-0.7786]]]])

In [13]:
def Norm2D(x):
    y = (x-torch.mean(x))/torch.sqrt(torch.var(x)+1e-5)
    return y
x = torch.randn(3,2)
y = Norm2D(x)
x,y

(tensor([[ 0.5407,  1.1947],
         [-0.5370,  0.6121],
         [-1.0116,  0.8733]]),
 tensor([[ 0.3044,  1.0644],
         [-0.9479,  0.3874],
         [-1.4994,  0.6910]]))